# Data Modelisation

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from gensim.models import Phrases
from gensim.corpora import Dictionary
from gensim.models import LdaModel, CoherenceModel
from gensim.models.LdaMulticore import LdaMulticore
import pyLDAvis.gensim
import pickle 
import pyLDAvis

## 1) Data preparation

In [2]:
data = pd.read_csv("datasets/posts_clean.csv")
data.head(3)

,Id,Title,Body,Tags
0,415192,good way create simple python web service,I use python year I little experience python w...,<python><web-services>
1,415344,log implementation prefer,I implement log class c try decide I curious k...,<debugging><language-agnostic><logging>
2,414981,directly modify list element,I struct struct map public int size public map...,<c#><.net>


In [3]:
# remove < and > around Tags
data["Tags"] = data["Tags"].replace({"<" : " "}, regex=True)
data["Tags"] = data["Tags"].replace({">" : " "}, regex=True)

In [4]:
data.head()

,Id,Title,Body,Tags
0,415192,good way create simple python web service,I use python year I little experience python w...,python web-services
1,415344,log implementation prefer,I implement log class c try decide I curious k...,debugging language-agnostic logging
2,414981,directly modify list element,I struct struct map public int size public map...,c# .net
3,415580,regex name group java,understanding java regex package not support n...,java regex
4,415753,instance cache objective c,I want cache instance certain class class keep...,objective-c weak-references


In [5]:
docs = data["Body"].to_list()

In [6]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
for idx in range(len(docs)):
    docs[idx] = docs[idx].lower()  # Convert to lowercase.
    docs[idx] = tokenizer.tokenize(docs[idx])  # Split into words.

# Remove numbers, but not words that contain numbers.
docs = [[token for token in doc if not token.isnumeric()] for doc in docs]

# Remove words that are only one character.
docs = [[token for token in doc if len(token) > 1] for doc in docs]

In [7]:
lemmatizer = WordNetLemmatizer()
docs = [[lemmatizer.lemmatize(token) for token in doc] for doc in docs]

In [8]:
# Compute bigrams.
# Add bigrams and trigrams to docs (only ones that appear 20 times or more).
bigram = Phrases(docs, min_count=20)
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [9]:
# Remove rare and common tokens.
# Create a dictionary representation of the documents.
dct = Dictionary(docs)
dct.filter_extremes(no_below=20, no_above=0.5)

In [10]:
# Bag-of-words representation of the documents.
corpus = [dct.doc2bow(doc) for doc in docs]

In [11]:
print('Number of unique tokens: %d' % len(dct))
print('Number of documents: %d' % len(corpus))

Number of unique tokens: 24935
Number of documents: 164598


## 2) Topic Modelling

In [12]:
# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dct[0]  # This is only to "load" the dictionary.
id2word = dct.id2token

model = LdaModel(
    corpus=corpus,
    id2word = id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

In [19]:
coherence_model_lda = CoherenceModel(model=model, texts=docs, dictionary=dct, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)


Coherence Score:  0.5933901185688508


In [33]:
def compute_coherence_values(corpus, dictionary, k, a, b):
    lda_model = LdaMulticore(corpus=corpus,
                        id2word = id2word,
                        chunksize=chunksize,
                        alpha=a,
                        eta=b,
                        iterations=iterations,
                        num_topics=k,
                        passes=passes,
                        eval_every=eval_every)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dct, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [34]:
from gensim.utils import ClippedCorpus
import gensim
import tqdm

In [36]:
grid = {}
grid['Validation_Set'] = {}
# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)
# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')
# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')
# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }
# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=dct, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

  1%|          | 6/540 [19:28<29:25:07, 198.33s/it]

In [ ]:
lda_model = LdaMulticore(corpus=corpus,
                        id2word=id2word,
                        num_topics=, 
                        random_state=100,
                        chunksize=100,
                        passes=10,
                        alpha=,
                        eta=)

In [ ]:
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared